In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import category_encoders as ce

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import LearningRateScheduler
from feature_engine.creation import CyclicalFeatures

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import numpy as np


## Helper Functions

In [3]:
def calculate_crs_deptime(deptime, depdelay):
    hours = deptime // 100
    minutes = deptime % 100
    total_minutes = (hours * 60 + minutes) - depdelay
    crs_hours = abs(total_minutes) // 60
    crs_minutes = total_minutes % 60
    if total_minutes < 0:
        return total_minutes
    return crs_hours * 100 + crs_minutes

In [4]:
def binary_encode(df, columns):
    encoder = ce.BinaryEncoder(cols=columns)

    # Fit and transform to produce binary encoded data
    df_encoded = encoder.fit_transform(df[columns])

    # Merge the encoded data back with the original DataFrame
    df = df.drop(columns, axis=1)
    df = pd.concat([df, df_encoded], axis=1)
    return df

In [5]:
def scale_columns_by_standard_scaler(x_train_df, x_validation_df, x_test_df, x_test_2023_df, columns):
    scaler = StandardScaler()
    
    x_train_df[columns] = scaler.fit_transform(x_train_df[columns])
    x_validation_df[columns] = scaler.transform(x_validation_df[columns])
    x_test_df[columns] = scaler.transform(x_test_df[columns])
    
    if x_test_2023_df is not None:
        x_test_2023_df[columns] = scaler.transform(x_test_2023_df[columns])
    
    return x_train_df, x_validation_df, x_test_df, x_test_2023_df

In [6]:
def scale_columns_by_cyclical_scaler(x_train_df, x_validation_df, x_test_df, x_test_2023_df, columns):
    scaler = CyclicalFeatures(drop_original=True)
    
    x_train_transformed_df = scaler.fit_transform(x_train_df[columns])
    x_train_df = pd.concat([x_train_df.drop(columns, axis=1), x_train_transformed_df], axis=1)
    
    x_validation_transformed_df = scaler.transform(x_validation_df[columns])
    x_validation_df = pd.concat([x_validation_df.drop(columns, axis=1), x_validation_transformed_df], axis=1)


    x_test_transformed_df = scaler.transform(x_test_df[columns])
    x_test_df = pd.concat([x_test_df.drop(columns, axis=1), x_test_transformed_df], axis=1)
    
    if x_test_2023_df is not None:
        x_test_2023_transformed_df = scaler.transform(x_test_2023_df[columns])
        x_test_2023_df = pd.concat([x_test_2023_df.drop(columns, axis=1), x_test_2023_transformed_df], axis=1)

    return x_train_df, x_validation_df, x_test_df, x_test_2023_df

In [7]:
def step_decay(epoch):
    lrate = 0.001  # Start with this learning rate
    if epoch > 25 and epoch <= 40:
        lrate = 0.0001 # Drop to this for epochs 3 and 4
    elif epoch > 40:
        lrate = 0.00001 # Drop to this after epoch 5
    return lrate

In [8]:
def filter_df_for_feature_and_encode(df, features, encoded_columns):
    flight_df = df[features]
    flight_df = flight_df.dropna()

    # Encoding combined categorical variables
    flight_df = binary_encode(flight_df, encoded_columns)
    flight_df = pd.get_dummies(flight_df, columns=['Carrier'])
    return flight_df

In [9]:
def split_X_y_and_2023(df: pd.DataFrame):
    flight_df_2023 = df[df['Year'] == 2023]
    flight_df = df[df['Year'] != 2023]

    X = flight_df.drop('ArrDel15', axis=1)
    y = flight_df['ArrDel15']

    X_test_2023 = flight_df_2023.drop('ArrDel15', axis=1)
    y_test_2023 = flight_df_2023['ArrDel15']

    return X, y, X_test_2023, y_test_2023

In [10]:
def calculate_model_metrics(model, X_test, y_test):
    # Get model predictions
    y_pred_proba = model.predict(X_test)

    # If your model outputs probabilities for both classes, keep the one for the positive class
    if y_pred_proba.shape[1] == 2:
        y_pred_proba = y_pred_proba[:, 1]

    # Convert probabilities to class labels (0 or 1)
    y_pred = np.where(y_pred_proba > 0.5, 1, 0)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

    f1 = f1_score(y_test, y_pred)
    print("F1 Score:", f1)
    
    auc = roc_auc_score(y_test, y_pred_proba)
    print("AUC Score:", auc)

## Preprocessing and loading

In [11]:
file_name = 'flight_test_data.csv'
file_path = f"../data/{file_name}"
df_2023 = pd.read_csv(file_path)

# Preprocess
# Add Column: aircraft_daily_flight_count
df_2023['Aircraft_Daily_Flight_Count'] = None 

df_2023.sort_values(by=['FlightDate', 'DepTime'], inplace=True)

# Group by 'flightdate' and 'Tail_Number' and use cumcount() to get a count within each group
df_2023['Aircraft_Daily_Flight_Count'] = df_2023.groupby(['FlightDate', 'Tail_Number']).cumcount() + 1

In [12]:
file_name = 'flight_data_large_balanced.csv'
file_path = f"../data/{file_name}"
df = pd.read_csv(file_path)

In [13]:
# Combine to get same encodings for training
df = pd.concat([df, df_2023])

## Model run feature set 1  Large Balanced

In [14]:
features = ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 
'Dest', 'Origin', 'Distance', 'Carrier',
'Full-time', 'Part-time', 'Grand Total', 'CRSDepTime', 'CRSArrTime', 'CRSElapsedTime',
'Aircraft_Daily_Flight_Count', 'ArrDel15']

In [15]:
flight_df = filter_df_for_feature_and_encode(df, features, ['Dest', 'Origin'])
X, y, X_test_2023, y_test_2023 = split_X_y_and_2023(flight_df)

In [16]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [17]:
# Transform with StandardScaler
standard_columns = ['Distance', 'Full-time', 'Part-time', 'Grand Total', 'Aircraft_Daily_Flight_Count']
X_train, X_validation, X_test, X_test_2023 = scale_columns_by_standard_scaler(X_train, X_validation, X_test, X_test_2023, standard_columns)

# Transform with CyclicalFeatures
time_columns =['Quarter', 'Month', 'DayofMonth', 'DayOfWeek'] 
# X_train, X_validation, X_test, X_test_2023 = scale_columns_by_cyclical_scaler(X_train, X_validation, X_test, X_test_2023, time_columns)

print(X_train.shape)

(4920530, 48)


In [18]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # or 'softmax' for multi-class classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lr_scheduler = LearningRateScheduler(step_decay)

history = model.fit(
    X_train, 
    y_train, 
    epochs=20, 
    batch_size=32, 
    validation_data=[X_validation, y_validation],
    callbacks=[lr_scheduler]
)

Epoch 1/20
153767/153767 [==============================] - 76s 492us/step - loss: 0.7023 - accuracy: 0.6421 - val_loss: 0.6304 - val_accuracy: 0.6508 - lr: 0.0010
Epoch 2/20
153767/153767 [==============================] - 76s 491us/step - loss: 0.6254 - accuracy: 0.6568 - val_loss: 0.6317 - val_accuracy: 0.6452 - lr: 0.0010
Epoch 3/20
153767/153767 [==============================] - 78s 507us/step - loss: 0.6238 - accuracy: 0.6584 - val_loss: 0.6238 - val_accuracy: 0.6609 - lr: 0.0010
Epoch 4/20
153767/153767 [==============================] - 77s 500us/step - loss: 0.6229 - accuracy: 0.6591 - val_loss: 0.6209 - val_accuracy: 0.6602 - lr: 0.0010
Epoch 5/20
153767/153767 [==============================] - 76s 495us/step - loss: 0.6225 - accuracy: 0.6595 - val_loss: 0.6214 - val_accuracy: 0.6616 - lr: 0.0010
Epoch 6/20
153767/153767 [==============================] - 73s 476us/step - loss: 0.6222 - accuracy: 0.6596 - val_loss: 0.6256 - val_accuracy: 0.6532 - lr: 0.0010
Epoch 7/20
15376

In [19]:
calculate_model_metrics(model, X_test, y_test)

32950/32950 [==============================] - 8s 240us/step
Accuracy: 0.6483943474962064
F1 Score: 0.5481802681915291
AUC Score: 0.6775547958696932


In [20]:
calculate_model_metrics(model, X_test_2023, y_test_2023)

2188/2188 [==============================] - 1s 241us/step
Accuracy: 0.5238
F1 Score: 0.15661370306649125
AUC Score: 0.6320750146938776


## Model run feature set 1 & 2 -- Large Balanced


In [21]:
features = ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 
'Dest', 'Origin', 'Distance', 'Carrier',
'Full-time', 'Part-time', 'Grand Total', 'CRSDepTime', 'CRSArrTime', 'CRSElapsedTime',
'Aircraft_Daily_Flight_Count', 'DepTime', 'DepDelay', 'TaxiOut', 'WheelsOff', 'ArrDel15']

In [22]:
flight_df = filter_df_for_feature_and_encode(df, features, ['Dest', 'Origin'])
X, y, X_test_2023, y_test_2023 = split_X_y_and_2023(flight_df)

In [23]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [24]:
# Transform with StandardScaler
standard_columns = ['Distance', 'Full-time', 'Part-time', 'Grand Total', 'Aircraft_Daily_Flight_Count']
X_train, X_validation, X_test, X_test_2023 = scale_columns_by_standard_scaler(X_train, X_validation, X_test, X_test_2023, standard_columns)

# Transform with CyclicalFeatures
time_columns =['Quarter', 'Month', 'DayofMonth', 'DayOfWeek'] 
X_train, X_validation, X_test, X_test_2023 = scale_columns_by_cyclical_scaler(X_train, X_validation, X_test, X_test_2023, time_columns)

print(X_train.shape)

(4920530, 56)


In [25]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # or 'softmax' for multi-class classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=10, 
    batch_size=32, 
    validation_data=[X_validation, y_validation]
)

Epoch 1/10
153767/153767 [==============================] - 80s 518us/step - loss: 0.2330 - accuracy: 0.9126 - val_loss: 0.2155 - val_accuracy: 0.9182
Epoch 2/10
153767/153767 [==============================] - 77s 502us/step - loss: 0.2197 - accuracy: 0.9163 - val_loss: 0.2142 - val_accuracy: 0.9182
Epoch 3/10
153767/153767 [==============================] - 76s 495us/step - loss: 0.2184 - accuracy: 0.9168 - val_loss: 0.2239 - val_accuracy: 0.9154
Epoch 4/10
153767/153767 [==============================] - 73s 477us/step - loss: 0.2176 - accuracy: 0.9171 - val_loss: 0.2186 - val_accuracy: 0.9163
Epoch 5/10
153767/153767 [==============================] - 78s 506us/step - loss: 0.2167 - accuracy: 0.9174 - val_loss: 0.2187 - val_accuracy: 0.9189
Epoch 6/10
153767/153767 [==============================] - 83s 538us/step - loss: 0.2163 - accuracy: 0.9176 - val_loss: 0.2222 - val_accuracy: 0.9145
Epoch 7/10
153767/153767 [==============================] - 77s 499us/step - loss: 0.2158 - ac

In [26]:
calculate_model_metrics(model, X_test, y_test)

32950/32950 [==============================] - 7s 213us/step
Accuracy: 0.9189776176024279
F1 Score: 0.8953163790716084
AUC Score: 0.9649571631399996


In [27]:
calculate_model_metrics(model, X_test_2023, y_test_2023)

2188/2188 [==============================] - 0s 216us/step
Accuracy: 0.9038
F1 Score: 0.9004641262896503
AUC Score: 0.9611291375510204


## Model run feature set 1 --  No weather | Small

In [28]:
file_name = 'flight_data_weather.csv'
file_path = f"../data/{file_name}"
df_weather = pd.read_csv(file_path)

df_weather['CRSDepTime'] = df_weather.apply(lambda x: calculate_crs_deptime(x['DepTime'], x['DepDelay']), axis=1)

In [29]:
features = ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 
'Dest', 'Origin', 'Distance', 'Carrier',
'Full-time', 'Part-time', 'Grand Total', 'CRSDepTime', 'CRSArrTime', 'CRSElapsedTime',
'Aircraft_Daily_Flight_Count', 'ArrDel15']

In [30]:
flight_df = df_weather[features]
flight_df = flight_df.dropna()

X = flight_df.drop('ArrDel15', axis=1)
y = flight_df['ArrDel15']

In [31]:
# Encoding categorical variables
X = binary_encode(X, ['Dest', 'Origin'])
X = pd.get_dummies(X, columns=['Carrier'])

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [32]:
# Transform with StandardScaler
standard_columns = ['Distance', 'Full-time', 'Part-time', 'Grand Total', 'Aircraft_Daily_Flight_Count']
X_train, X_validation, X_test, _ = scale_columns_by_standard_scaler(X_train, X_validation, X_test, None, standard_columns)

# Transform with CyclicalFeatures
time_columns =['Quarter', 'Month', 'DayofMonth', 'DayOfWeek'] 
X_train, X_validation, X_test, _ = scale_columns_by_cyclical_scaler(X_train, X_validation, X_test, None, time_columns)

print(X_train.shape)

(240184, 52)


In [33]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # or 'softmax' for multi-class classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lr_scheduler = LearningRateScheduler(step_decay)

history = model.fit(
    X_train, 
    y_train, 
    epochs=50, 
    batch_size=32, 
    validation_data=[X_validation, y_validation],
    callbacks=[lr_scheduler]
)

Epoch 1/50
7506/7506 [==============================] - 4s 474us/step - loss: 1.4724 - accuracy: 0.5256 - val_loss: 0.7856 - val_accuracy: 0.5143 - lr: 0.0010
Epoch 2/50
7506/7506 [==============================] - 3s 464us/step - loss: 0.8682 - accuracy: 0.5341 - val_loss: 0.7034 - val_accuracy: 0.5293 - lr: 0.0010
Epoch 3/50
7506/7506 [==============================] - 4s 471us/step - loss: 0.6943 - accuracy: 0.5538 - val_loss: 0.6799 - val_accuracy: 0.5606 - lr: 0.0010
Epoch 4/50
7506/7506 [==============================] - 3s 465us/step - loss: 0.6797 - accuracy: 0.5625 - val_loss: 0.6772 - val_accuracy: 0.5658 - lr: 0.0010
Epoch 5/50
7506/7506 [==============================] - 3s 465us/step - loss: 0.6780 - accuracy: 0.5658 - val_loss: 0.6765 - val_accuracy: 0.5675 - lr: 0.0010
Epoch 6/50
7506/7506 [==============================] - 4s 468us/step - loss: 0.6771 - accuracy: 0.5684 - val_loss: 0.6778 - val_accuracy: 0.5641 - lr: 0.0010
Epoch 7/50
7506/7506 [========================

In [34]:
calculate_model_metrics(model, X_test, y_test)

1609/1609 [==============================] - 0s 219us/step
Accuracy: 0.5868889407010182
F1 Score: 0.5802503257393297
AUC Score: 0.6240792844753368


## Model run feature set 1  weather | Small

In [35]:
file_name = 'flight_data_weather.csv'
file_path = f"../data/{file_name}"
df_weather = pd.read_csv(file_path)

df_weather['CRSDepTime'] = df_weather.apply(lambda x: calculate_crs_deptime(x['DepTime'], x['DepDelay']), axis=1)

In [36]:
features = ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 
'Dest', 'Origin', 'Distance', 'Carrier',
'Full-time', 'Part-time', 'Grand Total', 'CRSDepTime', 'CRSArrTime', 'CRSElapsedTime',
'Origin_Windspeed', 'Origin_Precip', 'Dest_Windspeed', 'Dest_Precip', 'dest_ianaTimeZone',
'origin_ianaTimeZone', 'Aircraft_Daily_Flight_Count', 'ArrDel15']

In [37]:
flight_df = df_weather[features]
flight_df = flight_df.dropna()

X = flight_df.drop('ArrDel15', axis=1)
y = flight_df['ArrDel15']

In [38]:
# Encoding categorical variables
X = binary_encode(X, ['origin_ianaTimeZone', 'dest_ianaTimeZone', 'Dest', 'Origin'])
X = pd.get_dummies(X, columns=['Carrier'])

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [39]:
# Transform with StandardScaler
standard_columns = ['Distance', 'Full-time', 'Part-time', 'Grand Total', 'Origin_Windspeed', 'Origin_Precip', 'Dest_Windspeed', 'Dest_Precip', 'Aircraft_Daily_Flight_Count']
X_train, X_validation, X_test, _ = scale_columns_by_standard_scaler(X_train, X_validation, X_test, None, standard_columns)

# Transform with CyclicalFeatures
time_columns =['Quarter', 'Month', 'DayofMonth', 'DayOfWeek'] 
X_train, X_validation, X_test, _ = scale_columns_by_cyclical_scaler(X_train, X_validation, X_test, None, time_columns)

print(X_train.shape)

(240184, 64)


In [40]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # or 'softmax' for multi-class classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lr_scheduler = LearningRateScheduler(step_decay)

history = model.fit(
    X_train, 
    y_train, 
    epochs=50, 
    batch_size=32, 
    validation_data=[X_validation, y_validation],
    callbacks=[lr_scheduler]
)

Epoch 1/50
7506/7506 [==============================] - 4s 493us/step - loss: 1.4196 - accuracy: 0.5277 - val_loss: 1.0109 - val_accuracy: 0.5178 - lr: 0.0010
Epoch 2/50
7506/7506 [==============================] - 4s 486us/step - loss: 0.8209 - accuracy: 0.5403 - val_loss: 0.7117 - val_accuracy: 0.5367 - lr: 0.0010
Epoch 3/50
7506/7506 [==============================] - 4s 483us/step - loss: 0.6941 - accuracy: 0.5555 - val_loss: 0.6763 - val_accuracy: 0.5682 - lr: 0.0010
Epoch 4/50
7506/7506 [==============================] - 4s 484us/step - loss: 0.6786 - accuracy: 0.5627 - val_loss: 0.6778 - val_accuracy: 0.5639 - lr: 0.0010
Epoch 5/50
7506/7506 [==============================] - 4s 483us/step - loss: 0.6762 - accuracy: 0.5678 - val_loss: 0.6772 - val_accuracy: 0.5696 - lr: 0.0010
Epoch 6/50
7506/7506 [==============================] - 4s 483us/step - loss: 0.6755 - accuracy: 0.5683 - val_loss: 0.6728 - val_accuracy: 0.5752 - lr: 0.0010
Epoch 7/50
7506/7506 [========================

In [41]:
calculate_model_metrics(model, X_test, y_test)

1609/1609 [==============================] - 0s 234us/step
Accuracy: 0.5874523976062797
F1 Score: 0.582561682886071
AUC Score: 0.6228084203082094


## Model run feature set 1 & 2 -  No Weather

In [42]:
file_name = 'flight_data_weather.csv'
file_path = f"../data/{file_name}"
df_weather = pd.read_csv(file_path)

df_weather['CRSDepTime'] = df_weather.apply(lambda x: calculate_crs_deptime(x['DepTime'], x['DepDelay']), axis=1)

In [43]:
features = ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 
'Dest', 'Origin', 'Distance', 'Carrier',
'Full-time', 'Part-time', 'Grand Total', 'CRSDepTime', 'CRSArrTime', 'CRSElapsedTime',
'Aircraft_Daily_Flight_Count', 
'DepDelay', 'DepTime', 'TaxiOut', 'WheelsOff', 'ArrDel15']

In [44]:
flight_df = df_weather[features]
flight_df = flight_df.dropna()

X = flight_df.drop('ArrDel15', axis=1)
y = flight_df['ArrDel15']

In [45]:
# Encoding categorical variables
X = binary_encode(X, ['Dest', 'Origin'])
X = pd.get_dummies(X, columns=['Carrier'])

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [46]:
# Transform with StandardScaler
standard_columns = ['Distance', 'Full-time', 'Part-time', 'Grand Total', 'Aircraft_Daily_Flight_Count']
X_train, X_validation, X_test, _ = scale_columns_by_standard_scaler(X_train, X_validation, X_test, None, standard_columns)

# Transform with CyclicalFeatures
time_columns =['Quarter', 'Month', 'DayofMonth', 'DayOfWeek'] 
X_train, X_validation, X_test, _ = scale_columns_by_cyclical_scaler(X_train, X_validation, X_test, None, time_columns)

print(X_train.shape)

(240184, 56)


In [47]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # or 'softmax' for multi-class classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lr_scheduler = LearningRateScheduler(step_decay)

history = model.fit(
    X_train, 
    y_train, 
    epochs=50, 
    batch_size=32, 
    validation_data=[X_validation, y_validation],
    callbacks=[lr_scheduler]
)

Epoch 1/50
7506/7506 [==============================] - 4s 496us/step - loss: 0.8493 - accuracy: 0.8481 - val_loss: 0.3911 - val_accuracy: 0.8589 - lr: 0.0010
Epoch 2/50
7506/7506 [==============================] - 4s 482us/step - loss: 0.3259 - accuracy: 0.8800 - val_loss: 0.2750 - val_accuracy: 0.8967 - lr: 0.0010
Epoch 3/50
7506/7506 [==============================] - 4s 482us/step - loss: 0.2685 - accuracy: 0.8944 - val_loss: 0.2691 - val_accuracy: 0.8925 - lr: 0.0010
Epoch 4/50
7506/7506 [==============================] - 4s 487us/step - loss: 0.2569 - accuracy: 0.8988 - val_loss: 0.2573 - val_accuracy: 0.8982 - lr: 0.0010
Epoch 5/50
7506/7506 [==============================] - 4s 482us/step - loss: 0.2540 - accuracy: 0.9000 - val_loss: 0.2504 - val_accuracy: 0.9006 - lr: 0.0010
Epoch 6/50
7506/7506 [==============================] - 4s 483us/step - loss: 0.2526 - accuracy: 0.9005 - val_loss: 0.2511 - val_accuracy: 0.9008 - lr: 0.0010
Epoch 7/50
7506/7506 [========================

In [48]:
calculate_model_metrics(model, X_test, y_test)

1609/1609 [==============================] - 0s 216us/step
Accuracy: 0.902599673583586
F1 Score: 0.8991611852030655
AUC Score: 0.9573190251783997


## Model run feature set 1 & 2 -- weather | Small

In [49]:
file_name = 'flight_data_weather.csv'
file_path = f"../data/{file_name}"
df_weather = pd.read_csv(file_path)

df_weather['CRSDepTime'] = df_weather.apply(lambda x: calculate_crs_deptime(x['DepTime'], x['DepDelay']), axis=1)

In [50]:
features = ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 
'Dest', 'Origin', 'Distance', 'Carrier',
'Full-time', 'Part-time', 'Grand Total', 'CRSDepTime', 'CRSArrTime', 'CRSElapsedTime',
'Origin_Windspeed', 'Origin_Precip', 'Dest_Windspeed', 'Dest_Precip', 'dest_ianaTimeZone',
'origin_ianaTimeZone', 'Aircraft_Daily_Flight_Count', 
'DepDelay', 'DepTime', 'TaxiOut', 'WheelsOff', 'ArrDel15']

In [51]:
flight_df = df_weather[features]
flight_df = flight_df.dropna()

X = flight_df.drop('ArrDel15', axis=1)
y = flight_df['ArrDel15']

In [52]:
# Encoding categorical variables
X = binary_encode(X, ['origin_ianaTimeZone', 'dest_ianaTimeZone', 'Dest', 'Origin'])
X = pd.get_dummies(X, columns=['Carrier'])

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [53]:
# Transform with StandardScaler
standard_columns = ['Distance', 'Full-time', 'Part-time', 'Grand Total', 'Origin_Windspeed', 'Origin_Precip', 'Dest_Windspeed', 'Dest_Precip', 'Aircraft_Daily_Flight_Count']
X_train, X_validation, X_test, _ = scale_columns_by_standard_scaler(X_train, X_validation, X_test, None, standard_columns)

# Transform with CyclicalFeatures
time_columns =['Quarter', 'Month', 'DayofMonth', 'DayOfWeek'] 
X_train, X_validation, X_test, _ = scale_columns_by_cyclical_scaler(X_train, X_validation, X_test, None, time_columns)

print(X_train.shape)

(240184, 68)


In [54]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # or 'softmax' for multi-class classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lr_scheduler = LearningRateScheduler(step_decay)

history = model.fit(
    X_train, 
    y_train, 
    epochs=50, 
    batch_size=32, 
    validation_data=[X_validation, y_validation],
    callbacks=[lr_scheduler]
)

Epoch 1/50
7506/7506 [==============================] - 4s 538us/step - loss: 0.7516 - accuracy: 0.8502 - val_loss: 0.3306 - val_accuracy: 0.8754 - lr: 0.0010
Epoch 2/50
7506/7506 [==============================] - 4s 551us/step - loss: 0.3216 - accuracy: 0.8805 - val_loss: 0.3037 - val_accuracy: 0.8843 - lr: 0.0010
Epoch 3/50
7506/7506 [==============================] - 4s 544us/step - loss: 0.2663 - accuracy: 0.8953 - val_loss: 0.2764 - val_accuracy: 0.8909 - lr: 0.0010
Epoch 4/50
7506/7506 [==============================] - 4s 530us/step - loss: 0.2566 - accuracy: 0.8989 - val_loss: 0.2532 - val_accuracy: 0.8988 - lr: 0.0010
Epoch 5/50
7506/7506 [==============================] - 4s 583us/step - loss: 0.2537 - accuracy: 0.8995 - val_loss: 0.2567 - val_accuracy: 0.8970 - lr: 0.0010
Epoch 6/50
7506/7506 [==============================] - 4s 555us/step - loss: 0.2515 - accuracy: 0.9009 - val_loss: 0.2567 - val_accuracy: 0.8998 - lr: 0.0010
Epoch 7/50
7506/7506 [========================

In [55]:
calculate_model_metrics(model, X_test, y_test)

1609/1609 [==============================] - 0s 223us/step
Accuracy: 0.903396285070335
F1 Score: 0.9000361896336805
AUC Score: 0.9591282072843723
